<a href="https://colab.research.google.com/github/makhijakabir/machine-learning/blob/main/NNHandWrittenRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Using Neural Networks to make a model that recognizes Hand Written Digits

##Imports

In [76]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [77]:
from numpy.random import seed
seed(888)
from tensorflow.random import set_seed
set_seed(404)

In [78]:
import os
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.6.0


##Constants

In [79]:
xTrainPath = '/content/drive/MyDrive/Colab Notebooks/imageClassifyData/digit_xtrain.csv'
xTestPath = '/content/drive/MyDrive/Colab Notebooks/imageClassifyData/digit_xtest.csv'
yTrainPath = '/content/drive/MyDrive/Colab Notebooks/imageClassifyData/digit_ytrain.csv'
yTestPath = '/content/drive/MyDrive/Colab Notebooks/imageClassifyData/digit_ytest.csv'

nrClasses = 10
validationSize = 10000

imageWidth = 28
imageHeight = 28
channels = 1
totalInputs = imageWidth * imageHeight * channels

##Get the Data

In [80]:
%%time

yTrainAll = np.loadtxt(yTrainPath, delimiter=',', dtype=int)
yTestAll = np.loadtxt(yTestPath, delimiter=',', dtype=int)

xTrainAll = np.loadtxt(xTrainPath, delimiter=',', dtype=int)
xTestAll = np.loadtxt(xTestPath, delimiter=',', dtype=int)

CPU times: user 51.6 s, sys: 1.83 s, total: 53.4 s
Wall time: 53.4 s


##Data Exploration

In [81]:
xTrainAll.shape

(60000, 784)

In [82]:
xTestAll.shape

(10000, 784)

##Data Pre Processing

In [83]:
#Re-Scale the features
xTrainAll, xTestAll = xTrainAll / 255.0, xTestAll / 255.0

###Conert the Target Dataset into one-hot encoding

In [84]:
yTrainAll = np.eye(nrClasses)[yTrainAll]
yTestAll = np.eye(nrClasses)[yTestAll]

In [85]:
yTrainAll.shape

(60000, 10)

###Create validaton Dataset from Training Dataset

In [86]:
xVal = xTrainAll[:validationSize]
yVal = yTrainAll[:validationSize]

xVal.shape

(10000, 784)

In [87]:
xTrain = xTrainAll[validationSize:]
yTrain = yTrainAll[validationSize:]

xTrain.shape

(50000, 784)

##Setup Tensorflow Graph

In [88]:
tf.compat.v1.disable_eager_execution()

X = tf.compat.v1.placeholder(tf.float32, shape=[None, totalInputs])
Y = tf.compat.v1.placeholder(tf.float32, shape=[None, nrClasses])

###Neural Network Architecture

####HyperParameters

In [89]:
nrEpochs = 5
learningRate = 0.0001

nHidden1 = 512
nHidden2 = 64

#####Hidden Layer 1

In [90]:
initialW1 = tf.random.truncated_normal(shape=[totalInputs, nHidden1], stddev=0.1, seed=42)
w1 = tf.Variable(initial_value=initialW1)

In [91]:
initialB1 = tf.constant(value=0.0, shape=[nHidden1])
b1 = tf.Variable(initial_value=initialB1)

In [92]:
layer1_in = tf.matmul(X, w1) + b1

In [93]:
layer1_out = tf.nn.relu(layer1_in)

#####Hidden Layer 2

In [104]:
initialW2 = tf.random.truncated_normal(shape=[nHidden1, nHidden2], stddev=0.1, seed=42)
w2 = tf.Variable(initial_value=initialW2)

initialB2 = tf.constant(value=0.0, shape=[nHidden2])
b2 = tf.Variable(initial_value=initialB2)

layer2_in = tf.matmul(layer1_out, w2) + b2
layer2_out = tf.nn.relu(layer2_in)

#####Output Layer

In [105]:
initialW3 = tf.random.truncated_normal(shape=[nHidden2, nrClasses], stddev=0.1, seed=42)
w3 = tf.Variable(initial_value=initialW3)

initialB3 = tf.constant(value=0.0, shape=[nrClasses])
b3 = tf.Variable(initial_value=initialB3)

layer3_in = tf.matmul(layer2_out, w3) + b3
output = tf.nn.softmax(layer3_in)